In [1]:
#Import important Libraries

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler  #library to normalize
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, auc, precision_recall_curve, roc_curve
from sklearn.linear_model import LogisticRegression   
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
import pandas as pd                     # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np                      # linear algebra
import seaborn as sns                   #  data visualization library 
from matplotlib import pyplot as plt    # plotting and visualization

In [2]:

df = pd.read_csv("salary.csv")        # Loading the csv file 

In [3]:
nRow, nCol = df.shape                # shape give the dimensions of data in thr form of rows and columns
print(nRow)
print(nCol)

32561
15


# Pre-processing

In [7]:
df=pd.DataFrame(df)          
df_1=df.columns.tolist()              # This convert the column in the form of list.
print(df_1)    

['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'salary']


In [8]:
df.head(5)                      #It helps to visualized the dataset

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39.0,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0,40.0,United-States,<=50K
1,50.0,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0,13.0,United-States,<=50K
2,38.0,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0,40.0,United-States,<=50K
3,53.0,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0,40.0,United-States,<=50K
4,28.0,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0,40.0,Cuba,<=50K


In [9]:
df.isnull().sum()       # This function gives the information about Null values against each features

age               3
workclass         3
fnlwgt            4
education         2
education-num     2
marital-status    2
occupation        5
relationship      4
race              2
sex               0
capital-gain      3
capital-loss      0
hours-per-week    1
native-country    2
salary            0
dtype: int64

In [10]:
from sklearn.preprocessing import LabelEncoder

def labelencoder(df):                                     #It convert the catorigcal and string data into numerical values to 
    for c in df.columns:                                        #interperate easily.
        if df[c].dtype=='object': 
            df[c] = df[c].fillna('N')
            lbl = LabelEncoder()
            lbl.fit(list(df[c].values))
            df[c] = lbl.transform(df[c].values)
    return df

In [11]:
data1=labelencoder(df)                            #calling function of labelencoder
data1                                             #Visualization of data after performing labelencoder  technique

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39.0,7,77516.0,9,13.0,4,1,1,4,1,2174.0,0,40.0,39,0
1,50.0,6,83311.0,9,13.0,2,4,0,4,1,0.0,0,13.0,39,0
2,38.0,4,215646.0,11,9.0,0,6,1,4,1,0.0,0,40.0,39,0
3,53.0,4,234721.0,1,7.0,2,6,0,2,1,0.0,0,40.0,39,0
4,28.0,4,338409.0,9,13.0,2,10,5,2,0,0.0,0,40.0,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27.0,4,257302.0,7,12.0,2,13,5,4,0,0.0,0,38.0,39,0
32557,40.0,4,154374.0,11,9.0,2,7,0,4,1,0.0,0,40.0,39,1
32558,58.0,4,151910.0,11,9.0,6,1,4,4,0,0.0,0,40.0,39,0
32559,22.0,4,201490.0,11,9.0,4,1,3,4,1,0.0,0,20.0,39,0


In [12]:

Labels=data1['salary']                        # It take salary column as a targeted label.
dataX=data1.drop('salary',axis=1)             #It drop the salary column and consider all other values .


# Handle  Missing Values or NAN values

In [10]:
from sklearn.impute import SimpleImputer                                         #SimpleImputer  help to filed the missing 
                                                                                   # or non-values  using means or median.
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(dataX)
X = imputer.transform(dataX)

# Feature Selection

In [11]:
#Bruta or RFE are used as a feature selection method, It select useful features and drop less useful features.

In [12]:
from sklearn.ensemble import RandomForestClassifier
model =RandomForestClassifier(max_depth=1) #For Boruta
from boruta import BorutaPy
feat_selector = BorutaPy(model, n_estimators='auto', verbose=1, random_state=101)
feat_selector.fit(X,Labels)
print(feat_selector.support_)  #Should we accept the featur
print(feat_selector.ranking_) #Rank 1 is the best
X_filtered1 = feat_selector.transform(X)  #Apply feature selection and return transformed data

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100
Iteration: 30 / 100
Iteration: 31 / 100
Iteration: 32 / 100
Iteration: 33 / 100
Iteration: 34 / 100
Iteration: 35 / 100
Iteration: 36 / 100
Iteration: 37 / 100
Iteration: 38 / 100
Iteration: 39 / 100
Iteration: 40 / 100
Iteration: 41 / 100
Iteration: 42 / 100
Iteration: 43 / 100
Iteration: 44 / 100
Iteration: 45 / 100
Iteration: 46 / 100
Iteration: 47 / 100
Iteration: 48 / 100
Iteration: 49 / 100
Iteration: 50 / 100
Iteration

In [13]:
Final_selected=X_filtered1

In [14]:
from sklearn.preprocessing import MinMaxScaler

#Final_selected =  (Final_selected - Final_selected.mean()) / Final_selected.std()
scaler = MinMaxScaler(feature_range=(0, 1))                        #It normalized the whole dataset between 0 and 1.
Final_selected = scaler.fit_transform(Final_selected)

In [15]:
#This step divided the whole dataset into training and testing by keeping test size 20%

X_train, X_test,y_train, y_test = train_test_split(Final_selected,Labels, test_size=0.20,random_state=9)


In [16]:
print('training features =',X_train.shape)
print('testing features   =',y_train.shape)
print('training labels=',X_test.shape)
print('testing labels =',y_test.shape)

training features = (26048, 14)
testing features   = (26048,)
training labels= (6513, 14)
testing labels = (6513,)


# RandomForest (RF)

In [17]:
#This is RandomClassifier , which is combination of multiple DT classifier .
#We used X_train and y_train data to trained this model 
#Perform prediction using X_test
# The accuracy and confusion matrix gives the detail information regarding  to classification output result

In [18]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100)       
model.fit(X_train, y_train)
prediction = model.predict(X_test)

In [19]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
print(classification_report(y_test, prediction))
print(accuracy_score(y_test, prediction))
print(confusion_matrix(y_test, prediction))

              precision    recall  f1-score   support

           0       0.89      0.93      0.91      4974
           1       0.73      0.62      0.67      1539

    accuracy                           0.86      6513
   macro avg       0.81      0.78      0.79      6513
weighted avg       0.85      0.86      0.85      6513

0.8567480423767849
[[4619  355]
 [ 578  961]]


# SVM

In [21]:
from sklearn.svm import SVC
from sklearn import svm
#model = svm.LinearSVC(multi_class="ovr")
model = svm.SVC(kernel='rbf', gamma=7.9, C=20, decision_function_shape='ovo')
model.fit(X_train, y_train)
prediction = model.predict(X_test)

In [22]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
print(classification_report(y_test, prediction))
print(accuracy_score(y_test, prediction))
print(confusion_matrix(y_test, prediction))

              precision    recall  f1-score   support

           0       0.88      0.91      0.89      4974
           1       0.68      0.59      0.63      1539

    accuracy                           0.84      6513
   macro avg       0.78      0.75      0.76      6513
weighted avg       0.83      0.84      0.83      6513

0.836480884385076
[[4536  438]
 [ 627  912]]


# Decision Tree (DT) 

In [23]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
prediction = model.predict(X_test)

In [24]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
print(classification_report(y_test, prediction))
print(accuracy_score(y_test, prediction))
print(confusion_matrix(y_test, prediction))

              precision    recall  f1-score   support

           0       0.88      0.88      0.88      4974
           1       0.61      0.62      0.62      1539

    accuracy                           0.82      6513
   macro avg       0.75      0.75      0.75      6513
weighted avg       0.82      0.82      0.82      6513

0.8183632734530938
[[4373  601]
 [ 582  957]]


# Gradient Boosting Machine(GBM) 


In [33]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(random_state=101)
model.fit(X_train, y_train)
prediction = model.predict(X_test)


In [34]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
print(classification_report(y_test, prediction))
print(accuracy_score(y_test, prediction))
print(confusion_matrix(y_test, prediction))

              precision    recall  f1-score   support

           0       0.88      0.95      0.92      4974
           1       0.79      0.59      0.68      1539

    accuracy                           0.87      6513
   macro avg       0.84      0.77      0.80      6513
weighted avg       0.86      0.87      0.86      6513

0.8674957776754184
[[4736  238]
 [ 625  914]]


# KNN

In [26]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=2)
model.fit(X_train, y_train)
prediction = model.predict(X_test)

In [27]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
print(classification_report(y_test, prediction))
print(accuracy_score(y_test, prediction))
print(confusion_matrix(y_test, prediction))

              precision    recall  f1-score   support

           0       0.83      0.95      0.89      4974
           1       0.69      0.39      0.50      1539

    accuracy                           0.81      6513
   macro avg       0.76      0.67      0.69      6513
weighted avg       0.80      0.81      0.79      6513

0.8137571011822509
[[4701  273]
 [ 940  599]]


# XGB

In [28]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)
prediction = model.predict(X_test)

e:\pythonenvironment\scl\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
e:\pythonenvironment\scl\lib\site-packages\pkg_resources\__init__.py:123: PkgResourcesDeprecationWarning: 47152a2e7b15bc0d08bbaa1fed3146c8d0d18922- is an invalid version and will not be supported in a future release
  warnings.warn(
e:\pythonenvironment\scl\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:52:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [29]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
print(classification_report(y_test, prediction))
print(accuracy_score(y_test, prediction))
print(confusion_matrix(y_test, prediction))

              precision    recall  f1-score   support

           0       0.90      0.94      0.92      4974
           1       0.76      0.66      0.71      1539

    accuracy                           0.87      6513
   macro avg       0.83      0.80      0.81      6513
weighted avg       0.87      0.87      0.87      6513

0.8713342545677875
[[4658  316]
 [ 522 1017]]


# MLP

In [30]:
from sklearn.neural_network import MLPClassifier
MLP = MLPClassifier(max_iter=1500,activation='relu', learning_rate_init=0.001,shuffle=True,
                    learning_rate='constant', beta_1=0.999, beta_2=0.9 , momentum=0.88,
                    power_t=0.9, solver='lbfgs', alpha=1e-6, random_state=101)

In [31]:
MLP.fit(X_train, y_train)
prediction = MLP.predict(X_test)

e:\pythonenvironment\scl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [32]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
print(classification_report(y_test, prediction))
print(accuracy_score(y_test, prediction))
print(confusion_matrix(y_test, prediction))

              precision    recall  f1-score   support

           0       0.89      0.93      0.91      4974
           1       0.73      0.61      0.67      1539

    accuracy                           0.86      6513
   macro avg       0.81      0.77      0.79      6513
weighted avg       0.85      0.86      0.85      6513

0.855980346998311
[[4630  344]
 [ 594  945]]
